In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit

# Define the decreasing exponential model
def decreasing_exponential_model(x, a, b, c):
    return a * np.exp(-b * x) + c

def visualize_revised_model(data, std_dev_data):
    plt.figure(figsize=(12, 8))  # Set the figure size outside the loop

    temperatures = data['Temp'].unique()
    colors = plt.cm.tab20(np.linspace(0, 0.2, len(temperatures)))  # Unique colors for each temperature
    markers = ['o', 's', '^', 'd', 'p', 'p', '*']  # Ensure unique markers

    for idx, (temp, color, marker) in enumerate(zip(temperatures, colors, markers)):
        temp_data = data[data['Temp'] == temp]
        std_dev = std_dev_data[std_dev_data['Temp'] == temp]['Original Ratio']

        # Error bars for actual data points
        plt.errorbar(temp_data['hPa'], temp_data['Original Ratio'], yerr=std_dev, fmt=marker, markersize=10, color=color, fillstyle="none", markeredgewidth=1.5, capsize=6, label=f'{temp}°C')

        # Fit decreasing exponential model
        try:
            params, covariance = curve_fit(decreasing_exponential_model, temp_data['hPa'], temp_data['Original Ratio'], p0=[1,0.01,1])
            x_model = np.linspace(temp_data['hPa'].min(), temp_data['hPa'].max(), 100)
            y_model = decreasing_exponential_model(x_model, *params)
            plt.plot(x_model, y_model, color=color)
        except RuntimeError:
            print(f"Exponential fit failed for temperature {temp}°C.")

    plt.xlabel('Oxygen Partial Pressure (hPa)', fontsize=25)
    plt.ylabel('Red/Green Intensity', fontsize=25)
    plt.grid(True, which='both', linestyle='--', linewidth=0.5, color='gray', alpha=0.7)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    plt.legend(fontsize=20)
    plt.savefig(f"C:/Users/au519867/OneDrive - Aarhus universitet/Documents/Ph.D/PhD - Manuscripts/Optode System - Method paper/Plots/Calibration_exponential_fit.png", dpi=300)
    plt.show()

# Load and process the data
file_path = 'E:/Temperature_compensation/RAW_Calibration_data_paper.xlsx'
data = pd.read_excel(file_path)
data = data.drop(data[data["Temp"]==5.4].index)
averaged_data = data.groupby(['Temp', 'hPa']).mean().reset_index()
std_dev_data = data.groupby(['Temp', 'hPa']).std().reset_index()

# Execute the visualization function
visualize_revised_model(averaged_data, std_dev_data)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

# Load the data from the Excel file
#file_path = 'E:/Calibrations_fixed.xlsx'  # Replace with your file path
#data = pd.read_excel(file_path)
#data = data.drop(data[data["Temp"]==5.4].index)
#data = predicted_data

# Define an empty DataFrame to store k and f values
k_f_values = pd.DataFrame(columns=['Temp', 'k', 'f'])

# Averaging the intensity ratios for replicate oxygen levels at each temperature
averaged_data = data.groupby(['Temp', 'hPa']).mean().reset_index()

# Normalizing the averaged data
averaged_data['Normalized Intensity Ratio'] = averaged_data.groupby('Temp')['Original Ratio'].transform(lambda x: x.iloc[0] / x)

# Define the modified Stern-Volmer equation
def modified_stern_volmer(x, k, f):
    return 1 / (f / (1 + k * x) + (1 - f))

# Fit the modified Stern-Volmer equation for each temperature
parameters_df = pd.DataFrame(columns=['Temp', 'k', 'f'])
initial_params = {'k': 0.165, 'f': 0.887}  # Initial parameter estimates

for temp in averaged_data['Temp'].unique():
    temp_data = averaged_data[averaged_data['Temp'] == temp]
    popt, _ = curve_fit(modified_stern_volmer, temp_data['Oxygen%'], temp_data['Normalized Intensity Ratio'], p0=[initial_params['k'], initial_params['f']], bounds=([0, 0], [np.inf, 1]), maxfev=10000)
    parameters_df = parameters_df.append({'Temp': temp, 'k': popt[0], 'f': popt[1]}, ignore_index=True)

# Fit polynomial models for k and f
model_k = make_pipeline(PolynomialFeatures(degree=2), LinearRegression())
model_k.fit(parameters_df['Temp'][:, np.newaxis], parameters_df['k'])

model_f = make_pipeline(PolynomialFeatures(degree=2), LinearRegression())
model_f.fit(parameters_df['Temp'][:, np.newaxis], parameters_df['f'])

# Define the function for prediction
def predict_intensity_ratio(temp, oxygen, model_k, model_f):
    k_temp = model_k.predict([[temp]])[0]
    f_temp = model_f.predict([[temp]])[0]
    return 1 / (f_temp / (1 + k_temp * oxygen) + (1 - f_temp))

# Define the function for visualization
def visualize_predictions_normalized(temperatures, oxygen_range, model_k, model_f, normalized_data):
    global k_f_values  # Add this line to reference the global variable

    plt.figure(figsize=(12, 8))

    # Viridis color scheme
    colors = plt.cm.tab20(np.linspace(0, 0.2, len(temperatures)))
    
    markers = ['o', 's', '^', 'd', 'p', 'p', '*']  # Ensure unique marker

    # Plot predicted data
    for temp, color in zip(temperatures, colors):
        predicted_ratios = [predict_intensity_ratio(temp, o2, model_k, model_f) for o2 in oxygen_range]
        plt.plot(oxygen_range, predicted_ratios, color=color)

    # Plot actual data with different shapes
    for temp, color, marker in zip(normalized_data['Temp'].unique(), colors, markers):
        temp_data = normalized_data[normalized_data['Temp'] == temp]
        k_temp = model_k.predict([[temp]])[0]
        f_temp = model_f.predict([[temp]])[0]
        plt.scatter(temp_data['Oxygen%'], temp_data['Normalized Intensity Ratio'], marker=marker, s=80, color=color, linewidth=2, linestyle='None', label=f'{temp}°C (Ksv={k_temp:.3f}, f={f_temp:.3f})')

        # Append k and f values to the DataFrame
        k_f_values = k_f_values.append({'Temp': temp, 'k': k_temp, 'f': f_temp}, ignore_index=True)

    plt.xlabel('Air Saturation (%)', fontsize=20)
    plt.ylabel('R0/R', fontsize=20)
    plt.grid(True, which='both', linestyle='--', linewidth=0.5, color='gray', alpha=0.7)
  #plt.title('Revised Model Predictions vs. Actual Data with Standard Deviation', fontsize=16)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.legend(fontsize=16)
    plt.savefig(f"C:/Users/au519867/OneDrive - Aarhus universitet/Documents/Ph.D/PhD - Manuscripts/Optode System - Method paper/Plots/Stern_volmer_modelled_fit.png", dpi=300)
    plt.show()

# Example use of visualization
example_temperatures = [0.9, 10, 15, 20, 25.3]  # Example temperatures in °C
oxygen_range_example = np.linspace(0, 100)  # Example range of oxygen concentrations
visualize_predictions_normalized(example_temperatures, oxygen_range_example, model_k, model_f, averaged_data)

# Print the k and f values DataFrame
print(k_f_values)



In [ ]:
# Create a DataFrame for the RO values
ro_values = averaged_data[averaged_data['hPa'] == 0][['Temp', 'Original Ratio']].rename(columns={'Original Ratio': 'RO'})

# Merge the RO values with the parameters_df to create the final table
final_table = pd.merge(ro_values, k_f_values, on='Temp')

# Display the table
print(final_table)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Initialize linear regression models
model_ro = LinearRegression()
model_k = LinearRegression()
model_f = LinearRegression()

# Assuming final_table is your DataFrame containing Temp, RO, k, and f
X = final_table[['Temp']]

# Perform linear regression
# RO
y_ro = final_table['RO']
model_ro.fit(X, y_ro)
predicted_ro = model_ro.predict(X)
r2_ro = r2_score(y_ro, predicted_ro)
fitting_function_ro = f'RO = {model_ro.coef_[0]:.3f} * x + {model_ro.intercept_:.3f}\nR² = {r2_ro:.3f}'

# k
y_k = final_table['k']
model_k.fit(X, y_k)
predicted_k = model_k.predict(X)
r2_k = r2_score(y_k, predicted_k)
fitting_function_k = f'k = {model_k.coef_[0]:.6f} * x + {model_k.intercept_:.6f}\nR² = {r2_k:.3f}'

# f
y_f = final_table['f']
model_f.fit(X, y_f)
predicted_f = model_f.predict(X)
r2_f = r2_score(y_f, predicted_f)
fitting_function_f = f'f = {model_f.coef_[0]:.4f} * x + {model_f.intercept_:.4f}\nR² = {r2_f:.3f}'

# Plotting
plt.figure(figsize=(18, 6))

# Plot RO values and regression line
plt.subplot(1, 3, 1)
plt.plot(final_table['Temp'], final_table['RO'], marker='o', linestyle='',  markersize=10, color='black', label='RO')
plt.plot(final_table['Temp'], predicted_ro, linestyle='-', color='red', label='Regression Line')
plt.xlabel('Temperature (°C)', fontsize=20)
plt.ylabel('Intensity Ratio at 0% Air.sat (R0)',fontsize=20)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.text(0.35, 0.95, fitting_function_ro, transform=plt.gca().transAxes, fontsize=16, verticalalignment='top')

# Plot k values and regression line
plt.subplot(1, 3, 2)
plt.plot(final_table['Temp'], final_table['k'], marker='s', linestyle='',  markersize=10, color='black', label='k')
plt.plot(final_table['Temp'], predicted_k, linestyle='-', color='red', label='Regression Line')
plt.xlabel('Temperature (°C)',fontsize=20)
plt.ylabel('Quenching constant (Ksv)',fontsize=20)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.text(0.05, 0.95, fitting_function_k, transform=plt.gca().transAxes, fontsize=16, verticalalignment='top')

# Plot f values and regression line
plt.subplot(1, 3, 3)
plt.plot(final_table['Temp'], final_table['f'], marker='^', markersize=10 ,linestyle='', color='black', label='f')
plt.plot(final_table['Temp'], predicted_f, linestyle='-', color='red', label='Regression Line')
plt.xlabel('Temperature (°C)',fontsize=20)
plt.ylabel('Quenchable fraction (f)',fontsize=20)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.text(0.35, 0.95, fitting_function_f, transform=plt.gca().transAxes, fontsize=16, verticalalignment='top')

plt.tight_layout()
plt.savefig(f"C:/Users/au519867/OneDrive - Aarhus universitet/Documents/Ph.D/PhD - Manuscripts/Optode System - Method paper/Plots/r0_k_f_values.png", dpi=300)
plt.show()
